In [1]:
pip install dash

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dash import Dash, html, dcc
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output


In [12]:
df = pd.read_csv('la_liga_stats.csv')
df

,Jugador,País,Posc,Edad,Equipo,PJ,Titular,Mín,Gls.,Ass,...,TA,TR,xG,xAG,Gls..1,Ast,G+A.1,xG.1,xAG.1,xG+xAG
0,İlkay Gündoğan,GER,CC,32.0,Barcelona,36,33,2994.0,5.0,9.0,...,5.0,0.0,7.3,10.3,0.15,0.27,0.42,0.22,0.31,0.53
1,Jules Koundé,FRA,DF,24.0,Barcelona,35,32,2899.0,1.0,2.0,...,4.0,0.0,2.0,2.2,0.03,0.06,0.09,0.06,0.07,0.13
2,Robert Lewandowski,POL,DL,34.0,Barcelona,35,32,2750.0,19.0,8.0,...,5.0,0.0,18.1,4.9,0.62,0.26,0.88,0.59,0.16,0.75
3,João Cancelo,POR,DF,29.0,Barcelona,32,29,2503.0,2.0,4.0,...,7.0,0.0,1.6,6.1,0.07,0.14,0.22,0.06,0.22,0.28
4,Marc-André ter Stegen,GER,PO,31.0,Barcelona,28,28,2520.0,0.0,0.0,...,3.0,0.0,0.0,0.1,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
776,Iker Álvarez,AND,PO,22.0,Villarreal,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
777,Daniel Budesca,ESP,DF,17.0,Villarreal,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
778,Abraham Rando,ESP,DF,22.0,Villarreal,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
779,Total del equipo,NaN,NaN,27.0,Villarreal,38,418,3420.0,64.0,47.0,...,110.0,8.0,55.6,40.5,1.68,1.24,2.92,1.46,1.07,2.53


In [19]:
# Aplicando DASH obtenemos este resultado 

# Crea una nueva aplicación Dash
app = Dash(__name__)

# Define algunos colores para usar en la aplicación
colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

# Lee los datos del archivo CSV y filtra las filas donde el jugador no es 'Total del equipo' ni 'Total del oponente'
df = pd.read_csv('la_liga_stats.csv')
df = df[(df['Jugador'] != 'Total del equipo') & (df['Jugador'] != 'Total del oponente')]

# Define la interfaz de usuario de la aplicación
app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
    # Un gráfico que se actualizará en función de los valores del control deslizante y el menú desplegable
    dcc.Graph(
        id='life-exp-vs-gdp',
    ), 
    html.Br(),
    # Un control deslizante que permite seleccionar el número mínimo de goles
    html.Label('Slider'),
    dcc.Slider(
        id='my-slider',
        min=0,
        max=int(df['Gls.'].max()),
        value=int(df['Gls.'].mean()),
        marks={i: str(i) for i in range(0, int(df['Gls.'].max())+1)},
    ), 
    
    # Un menú desplegable que permite seleccionar uno o varios equipos
    html.Div([
        html.Label('Dropdown'),
        dcc.Dropdown(
            id='my-dropdown',
            options=[{'label': i, 'value': i} for i in df['Equipo'].unique()],
            value=[df['Equipo'].iloc[0]],  # El valor inicial es una lista con el primer equipo en tu DataFrame
            multi=True  # Permite múltiples selecciones
        ),
    ]),
    
    
])

# Define una función de devolución de llamada que se activa cuando cambian los valores del control deslizante o el menú desplegable
@app.callback(
    Output('life-exp-vs-gdp', 'figure'),
    [Input('my-slider', 'value'),
     Input('my-dropdown', 'value')])
def update_graph(value, selected_teams):
    # Filtra el DataFrame para incluir solo las filas donde los goles son mayores o iguales al valor seleccionado en el control deslizante y el equipo está en la lista de equipos seleccionados
    filtered_df = df[(df['Gls.'] >= value) & (df['Equipo'].isin(selected_teams))]
    # Crea un gráfico de dispersión con los datos filtrados
    fig = px.scatter(filtered_df, x="Gls.", y="Ass",
                     size="Gls.", color="Equipo", hover_name="Jugador",
                     log_x=True, size_max=60)
    # Actualiza el diseño del gráfico para usar los colores definidos anteriormente
    fig.update_layout(
        plot_bgcolor=colors['background'],
        paper_bgcolor=colors['background'],
        font_color=colors['text']
    )
    # Devuelve el gráfico para que se muestre en la interfaz de usuario
    return fig

# Ejecuta la aplicación
if __name__ == '__main__':
    app.run(debug=True)
